<a href="https://colab.research.google.com/github/FrancescoRosa3/Prova/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSEGNA GRUPPO 19
*   Demetrio Trimarco
*   Emilio Sorrentino
*   Francesco Rosa
*   Francesco Sabbarese

# SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

## IMPORT


In [ ]:
%load_ext tensorboard
from tensorflow import keras
%tensorflow_version 2.x
import tensorflow as tf
from keras.callbacks import TensorBoard
from datetime import datetime
from packaging import version
import os
import io
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
import numpy
import cv2
import json

In [ ]:
# import functions for pre processing and data augmentation
!cp /content/gdrive/MyDrive/CONSEGNA/TRAIN/corruptions.py .
!cp /content/gdrive/MyDrive/CONSEGNA/TRAIN/augmentation_transforms.py .
!cp /content/gdrive/MyDrive/CONSEGNA/TRAIN/dataset_tools.py .
import dataset_tools
!apt-get install libmagickwand-dev
!pip install Wand
import corruptions
import augmentation_transforms

# import vggface models
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_vggface
!pip install keras_applications
from keras_vggface.vggface import VGGFace
import keras_vggface.utils

## VARIABLES

In [ ]:
BASE_PATH = "/content/gdrive/MyDrive/CONSEGNA/"
DIRECTORY_TRAIN = "TRAIN/"
FOLDER_CHECKPOINT = BASE_PATH + DIRECTORY_TRAIN + "MODELS/model.{epoch:02d}- {val_loss:.2f}.h5"
TENSOR_BOARD_PATH = BASE_PATH + DIRECTORY_TRAIN + "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
NUM_EPOCHS = 50
BASE_MODEL = "resnet50"
WEIGHTS = "vggface"

BATCH_SIZE = 64
TRAIN_SIZE = 435000
VAL_SIZE = 140173

In [ ]:
# datasets paths
train_tfrecord_file_name = "/content/gdrive/MyDrive/CONSEGNA/DATASETS/SUBSET_2_train.tfrecord"
train_tfrecord_utkf_pt1_file_name =  "/content/gdrive/MyDrive/CONSEGNA/DATASETS/UTKFACE_train_pt1.tfrecord"
train_tfrecord_utkf_pt2_file_name = "/content/gdrive/MyDrive/CONSEGNA/DATASETS/UTKFACE_train_pt2.tfrecord"
train_tfrecord_appa_file_name = "/content/gdrive/MyDrive/CONSEGNA/DATASETS/APPA-REAL_train.tfrecord"
val_tfrecord_file_name = "/content/gdrive/MyDrive/CONSEGNA/DATASETS/SUBSET_2_val.tfrecord"

# RUN TRAINING

## Ground truth labels dictionary creation

In [ ]:
# create labels dictionary for SUBSET_2

from csv import reader

labels_csv_filename = '/content/gdrive/MyDrive/CONSEGNA/DATASETS/train.age_detected.csv'

def create_label_dictionary():
    temp_dict = {}
    with open(labels_csv_filename, 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            age = str(row[-1])
            temp_dict[row[0]] = age
    return temp_dict

labels_dictionary = create_label_dictionary()
print("Labels obtained")

Labels obtained


## DataAugmentation Classes

In [ ]:
from dataset_tools import get_random_eraser

class CropoutAugmentation():
    def __init__(self):
        self.eraser = get_random_eraser()
    def before_cut(self, img):
        return img
    def after_cut(self, img):
        return self.eraser(img)

class DefaultAugmentation():
    def before_cut(self, frame, roi):
        frame = random_image_rotate(frame, roi_center(roi))
        frame = random_image_skew(frame, roi_center(roi))
        return frame
    def augment_roi(self, roi):
        roi = random_change_roi(roi)
        roi = enclosing_square(roi)
        return roi
        
    def after_cut(self, img):
        img = random_brightness_contrast(img)
        img = random_flip(img)
        return img

class VGGFace2Augmentation():
    def before_cut(self, frame, roi):
        frame = random_monochrome(frame, random_fraction_yes=0.2)
        return frame
    def augment_roi(self, roi):
        roi= add_margin(roi, 0.3)
        roi = random_fixed_size_roi(roi, original_size=(256,256), dst_size=(224,224))
        return roi
    def after_cut(self, img):
        img = random_flip(img)
        return img

import numpy as np
def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]
    
    return crop

## Parse tfrecord

In [ ]:
import random

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [240, 240])
    image = tf.cast(image, tf.uint8)
    return image

def get_age_vggface2(filename):
  return round(float(labels_dictionary[filename]))

def get_age_utkf(filename):
  return round(float(str(filename).split("_",1)[0]))

def get_age(filename):
  filename = str(filename).split("'",2)[1]
  if str(filename).split(".")[-2] == "chip":
    return get_age_utkf(filename)
  else:
    return get_age_vggface2(filename)

def conv_normalize(image):
  return keras_vggface.utils.preprocess_input(image, 'channels_last', version=2)

def conv_BGR2RGB(image):
  return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def conv_RGB2BGR(image):
  return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

def apply_transform(image):

  choice = random.getrandbits(1)
  if choice == 1:
    crp = CropoutAugmentation()
    image = crp.after_cut(image)

  choice = random.getrandbits(1)
  if choice == 1:
    image = dataset_tools.random_flip(image)

  choice = random.getrandbits(1)
  if choice == 1:
    image = dataset_tools.random_brightness_contrast(image)

  choice = random.getrandbits(1)
  if choice == 1:
    image = get_random_crop(image, 150, 150)
    dim = (240, 240)
    # resize image
    image = cv2.resize(image, dim)
  
  image = tf.cast(image, tf.uint8)
  return image

def DataAugmentation(image):
  choice = np.random.choice([True, False], size=1, replace=False, p=[0.30, 0.70])
  if choice[0] == True:
    return apply_transform(image)
  else:
    return image

# parse tfrecord samples
def read_tfrecord(example, labeled, train, ds_type):
    features = (
        {
          "image": tf.io.FixedLenFeature([], dtype=tf.string),
          "filename": tf.io.FixedLenFeature([], dtype=tf.string),
          "label": tf.io.FixedLenFeature([], dtype=tf.int64)
        } if ds_type == "appa"
        else{
          "image": tf.io.FixedLenFeature([], dtype=tf.string),
          "filename": tf.io.FixedLenFeature([], dtype=tf.string)  
        }
    )

    example = tf.io.parse_single_example(example, features)

    image = example["image"]
    
    image = decode_image(image) # the decoded image has BGR channels
    
    # Apply data augmentation
    if train:
      image = tf.numpy_function(DataAugmentation, [image], tf.uint8)
    
    # Normalize the image (substract mean)
    image = tf.numpy_function(conv_BGR2RGB, [image], tf.uint8)
    image = tf.cast(image, dtype=tf.float32)

    # conv_normalize needs RGB images as input and returns BGR images as output
    image = tf.numpy_function(conv_normalize, [image], tf.float32)

    filename = example["filename"]

    
    if labeled:
      if ds_type != "appa":
        label = tf.numpy_function(get_age, [filename], tf.int64)
      else:
        label = example["label"]
      return image, label
    return image 

## Loading the training set

In [ ]:
def load_dataset(tfrecord_filename, labeled=True, train = True, ds_type = "vggface2"):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        tfrecord_filename
    )
    
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    
    dataset = dataset.map(
        partial(read_tfrecord, labeled = labeled, train = train, ds_type = ds_type), num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

## Pipeline creation function

In [ ]:
# Concatanates batches
def concat(*ds_elements):
    #Create one empty list for each component of the dataset
    lists = [[] for _ in ds_elements[0]]
    for element in ds_elements:
        for i, tensor in enumerate(element):
            #For each element, add all its component to the associated list
            lists[i].append(tensor)
    #Concatenate each component list
    return tuple(tf.concat(l, axis=0) for l in lists)

# Create a dataset pipeline composed by:
#   - shuffling of data
#   - creation of batches
#   - prefatch of data to speed up the training
def apply_pipeline(paths, labeled = True, train = True):

    # Training set must be composed by data coming from VGGFace2, UTKFace and APPA-REAL datasets
    # Every batch is composed by 52 images from VGGFACE2, 8 images coming from UTKFace and 4 images from APPA-REAL  
    if train:
      vf2_dataset = load_dataset(paths[0], train=train)
      utkf_dataset1 = load_dataset(paths[1], train=train, ds_type="utkf")
      utkf_dataset2 = load_dataset(paths[2], train=train, ds_type="utkf")
      appa_dataset = load_dataset(paths[3], train=train, ds_type = "appa")
      
      zipped_ds = tf.data.Dataset.zip((vf2_dataset.shuffle(BATCH_SIZE*5).batch(52),
                                      utkf_dataset1.shuffle(BATCH_SIZE).repeat().batch(4),
                                      utkf_dataset2.shuffle(BATCH_SIZE).repeat().batch(4),
                                      appa_dataset.shuffle(BATCH_SIZE).repeat().batch(4)))
      unbalanced_batches_ds = zipped_ds.map(concat)
      dataset = unbalanced_batches_ds.repeat()
      dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    # Validation set is composed only by data coming from VGGFace2 dataset
    else:
      dataset = load_dataset(paths[0], labeled = labeled, train = train)
      dataset = dataset.shuffle(BATCH_SIZE*5)
      dataset = dataset.batch(BATCH_SIZE)
      dataset = dataset.repeat()
      dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

## Fit callback

In [ ]:
# tensorboard callback
class LRTensorBoard(TensorBoard):

  def __init__(self, log_dir, **kwargs):
    super().__init__(log_dir, **kwargs)
    self.lr_writer = tf.summary.create_file_writer(self.log_dir + '/learning')

  def on_epoch_end(self, epoch, logs=None):
    lr = getattr(self.model.optimizer, 'lr', None)
    with self.lr_writer.as_default():
      summary = tf.summary.scalar('learning_rate', lr, epoch)

    super().on_epoch_end(epoch, logs)

  def on_train_end(self, logs=None):
    super().on_train_end(logs)
    self.lr_writer.close()

tensorboard_callback = LRTensorBoard(log_dir=TENSOR_BOARD_PATH)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    FOLDER_CHECKPOINT,
    save_best_only=False,
    verbose = 1,
    save_freq = 'epoch',
    monitor='val_loss',
    mode = 'min'
)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, min_delta = 0.01, verbose = 1, mode = 'min', restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=3, min_delta = 0.1, mode = 'min', min_lr=0.0000001, verbose = 1
)

## Model creation

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, LeakyReLU, PReLU
from tensorflow.keras.initializers import Constant

def create_dense(num_neurons, x):
    x = BatchNormalization()(x)
    x = Dense(num_neurons)(x)
    x = PReLU(alpha_initializer=Constant(value=0.6), alpha_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.01))(x)
    x = Dropout(.2)(x)
    x = BatchNormalization()(x)
    x = Dense(num_neurons)(x)
    x = PReLU(alpha_initializer=Constant(value=0.6), alpha_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.01))(x)
    x = Dropout(.2)(x)
    x = BatchNormalization()(x)
    predictions = Dense(1, activation='relu')(x)
    return predictions

def make_model():
  
  # creation of the base pre-trained model
  base_model = VGGFace(include_top=False, input_shape=(240, 240, 3), pooling='avg', model=BASE_MODEL, weights = WEIGHTS)
  x = base_model.output
  last_layer = create_dense(2048, x)
  model = Model(inputs=base_model.input, outputs=last_layer)
  
  # set the internal layers to be trainable
  for layer in base_model.layers:
      layer.trainable = True

  # compile the model
  model.compile(
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.9),
      loss=tf.keras.losses.MeanAbsoluteError()
  )

  model.summary()

  return model

In [ ]:
model = make_model()

## FIT

In [ ]:
%tensorboard --logdir /content/gdrive/MyDrive/CONSEGNA/TRAIN/logs/scalars

In [ ]:
# datasets pipeline creation
train_dataset = apply_pipeline([train_tfrecord_file_name, train_tfrecord_utkf_pt1_file_name, train_tfrecord_utkf_pt2_file_name, train_tfrecord_appa_file_name], train = True)
validation_dataset = apply_pipeline([val_tfrecord_file_name], train = False)

# -------------------
tf.config.run_functions_eagerly(True)

history = model.fit(
    train_dataset,
    epochs=NUM_EPOCHS,
    validation_data=validation_dataset,
    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr, tensorboard_callback],
    steps_per_epoch = TRAIN_SIZE/BATCH_SIZE,
    verbose = 1,
    batch_size = BATCH_SIZE,
    validation_steps = VAL_SIZE/BATCH_SIZE
)